In [1]:
import os
os.getcwd()
os.chdir('c:\\Users\\larry\\Desktop\\Geoguessr ML Proj\\Geolocation-Project\\reverse_geocode3')

In [3]:
import data_loader
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
from unidecode import unidecode

In [4]:
df, _ = data_loader.load_cities_and_build_kdtree_from_geonames()
len(df)

Loading city data from GeoNames cities500.txt...
Reading natural_earth_data/cities500.txt ...
Loaded 215735 entries from GeoNames.

--- Loading Admin Name Mappings ---
Loading Admin1 names from natural_earth_data/admin1CodesASCII.txt...
Loaded 3893 Admin1 name mappings.
Loading Admin2 names from natural_earth_data/admin2Codes.txt...
Loaded 47356 Admin2 name mappings.

--- Mapping Admin Codes to Names ---
Admin1 names mapped.
Admin2 names mapped.
Converting GeoNames data to GeoDataFrame...
Building KDTree for GeoNames cities...
KDTree built successfully.


215735

In [5]:
df2 = pd.read_csv('natural_earth_data/worldcities.csv')
df2['population'] = df2['population'].fillna(0)
df2.loc[14, "city"] = "New York City"
df2.loc[14, "city_ascii"] = "New York City"
len(df2)

48059

In [6]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from unidecode import unidecode
from sklearn.cluster import OPTICS

def normalize_text(text):
    """Converts a string to its closest ASCII representation."""
    if isinstance(text, str):
        return unidecode(text)
    return text

def cluster_cities_with_optics(
    gdf_geonames, 
    df_worldcities, 
    max_eps_meters=20000, 
    min_samples=5, 
    xi=0.05
):
    """
    Identifies metropolitan areas by clustering cities using the OPTICS algorithm.

    This function replaces a fixed-radius search with a density-based approach
    to more accurately group boroughs and suburbs with their parent cities.

    Args:
        gdf_geonames (gpd.GeoDataFrame): Your primary GeoDataFrame of cities (your 'df').
                                         Must include columns: 'city_name', 'city_latitude',
                                         'city_longitude', 'city_population', 'geometry'.
        df_worldcities (pd.DataFrame): Your secondary DataFrame of major cities (your 'df2').
                                       Must include columns: 'city_ascii', 'lat',
                                       'lng', 'population', 'id'.
        max_eps_meters (int): The maximum distance (in meters) between two samples for them
                              to be considered as in the same neighborhood. This is an
                              upper bound for OPTICS performance, not a strict radius.
        min_samples (int): The number of samples in a neighborhood for a point to be
                           considered as a core point. This is the primary density parameter.
        xi (float): Determines the minimum steepness on the reachability plot that 
                    constitutes a cluster boundary. Between 0 and 1.

    Returns:
        gpd.GeoDataFrame: The original gdf_geonames with two new columns:
                          'metro_id' (the 'id' from df2 of the representative city) and
                          'metro_name' (the name of the representative city).
    """
    print("--- Starting OPTICS Clustering Process ---")

    # --- Step 1: Prepare and Combine Data ---
    print("Step 1: Preparing and standardizing data from both sources...")

    # Standardize GeoNames data (your 'df')
    gdf_geonames_std = gdf_geonames[['city_name', 'city_latitude', 'city_longitude', 'city_population', 'geometry']].copy()
    # Create a temporary unique ID from the index to merge back later
    gdf_geonames_std['original_id'] = gdf_geonames.index
    # Rename columns to a standard internal format
    gdf_geonames_std.rename(columns={
        'city_latitude': 'latitude',
        'city_longitude': 'longitude',
        'city_population': 'population'
    }, inplace=True)
    gdf_geonames_std['source'] = 'geonames'
    
    # Standardize and create GeoDataFrame for worldcities (your 'df2')
    df_worldcities_std = df_worldcities.copy()
    # Rename columns to the same standard internal format
    df_worldcities_std.rename(columns={
        'city_ascii': 'city_name',
        'lat': 'latitude',
        'lng': 'longitude',
        'id': 'original_id' # Use the worldcities ID as its unique identifier
    }, inplace=True)
    df_worldcities_std['source'] = 'worldcities'
    
    gdf_worldcities_std = gpd.GeoDataFrame(
        df_worldcities_std,
        geometry=gpd.points_from_xy(df_worldcities_std.longitude, df_worldcities_std.latitude),
        crs="EPSG:4326"
    )

    # Combine into a single GeoDataFrame using standardized columns
    cols_to_combine = ['original_id', 'city_name', 'latitude', 'longitude', 'population', 'geometry', 'source']
    gdf_all_cities = pd.concat([gdf_geonames_std[cols_to_combine], gdf_worldcities_std[cols_to_combine]], ignore_index=True)
    
    # Drop duplicates, keeping the one from worldcities if a conflict exists (as it's curated)
    gdf_all_cities.drop_duplicates(subset=['city_name', 'latitude', 'longitude'], keep='last', inplace=True)
    print(f"  Combined dataset has {len(gdf_all_cities)} unique cities.")

    # --- Step 2: Project Data and Run OPTICS ---
    print("\nStep 2: Projecting data and running OPTICS clustering...")
    print(f"  Parameters: max_eps={max_eps_meters}m, min_samples={min_samples}, xi={xi}")
    
    gdf_all_cities_proj = gdf_all_cities.to_crs(epsg=3857)
    coords = np.array(list(zip(gdf_all_cities_proj.geometry.x, gdf_all_cities_proj.geometry.y)))
    
    optics = OPTICS(min_samples=min_samples, max_eps=max_eps_meters, xi=xi, n_jobs=-1)
    optics.fit(coords)
    
    gdf_all_cities['cluster_id'] = optics.labels_
    num_clusters = len(np.unique(optics.labels_[optics.labels_ != -1]))
    num_noise = np.sum(optics.labels_ == -1)
    print(f"  OPTICS found {num_clusters} clusters and {num_noise} noise points.")

    # --- Step 3: Identify Representative City for Each Cluster ---
    print("\nStep 3: Identifying the most populous representative city for each cluster...")
    
    gdf_all_cities['population'] = pd.to_numeric(gdf_all_cities['population'], errors='coerce').fillna(0)
    
    # Find the representative (most populous city) for each cluster_id
    representatives = gdf_all_cities[gdf_all_cities['cluster_id'] != -1].loc[
        gdf_all_cities.groupby('cluster_id')['population'].idxmax()
    ]
    
    # Create a map from cluster_id to the representative's name and ID
    representatives_map_df = representatives[['cluster_id', 'city_name', 'original_id']].copy()
    representatives_map_df.rename(columns={'city_name': 'metro_name', 'original_id': 'metro_id'}, inplace=True)
    
    # --- Step 4: Map Representatives Back to Original GeoNames Data ---
    print("\nStep 4: Merging cluster results back into the original 'df' DataFrame...")
    
    # First, add the original_id to the main combined list so we can find it
    gdf_all_cities_with_id = gdf_all_cities.merge(gdf_geonames_std[['original_id', 'geometry']], on='geometry')

    # Merge the cluster information back to the main city list
    gdf_all_cities_merged = gdf_all_cities_with_id.merge(
        representatives_map_df, on='cluster_id', how='left'
    )
    
    # For noise points (not in a cluster), they represent themselves
    is_noise = gdf_all_cities_merged['metro_name'].isna()
    gdf_all_cities_merged.loc[is_noise, 'metro_name'] = gdf_all_cities_merged.loc[is_noise, 'city_name']
    gdf_all_cities_merged.loc[is_noise, 'metro_id'] = gdf_all_cities_merged.loc[is_noise, 'original_id_x'] # Use its own id
    
    # Filter for the original data source and the columns we need to add back
    final_cols_to_merge = gdf_all_cities_merged[gdf_all_cities_merged['source'] == 'geonames'][['original_id_x', 'metro_id', 'metro_name']]
    final_cols_to_merge.rename(columns={'original_id_x': 'original_id'}, inplace=True)
    final_cols_to_merge.drop_duplicates(subset=['original_id'], inplace=True)

    # Merge the new metro columns back to the original gdf_geonames using the index
    gdf_final = gdf_geonames.merge(
        final_cols_to_merge,
        left_index=True,
        right_on='original_id',
        how='left'
    )
    gdf_final.drop(columns=['original_id'], inplace=True, errors='ignore')

    print("\n--- OPTICS Process Complete! ---")
    print(f"Final dataset contains {len(gdf_final)} cities, enriched with metro area information.")
    
    return gdf_final

# --- Example Usage ---
# Assuming 'df' and 'df2' are pre-loaded with the specified column structures.

final_clustered_gdf = cluster_cities_with_optics(
    gdf_geonames=df, 
    df_worldcities=df2
)

# print("\nExample: Results for New York City Area")
# Check for a major city name that exists in your df2 data
nyc_area = final_clustered_gdf[final_clustered_gdf['metro_name'] == 'New York'] 
print(nyc_area[['city_name', 'metro_name']].head(10))

--- Starting OPTICS Clustering Process ---
Step 1: Preparing and standardizing data from both sources...
  Combined dataset has 263629 unique cities.

Step 2: Projecting data and running OPTICS clustering...
  Parameters: max_eps=20000m, min_samples=5, xi=0.05


KeyboardInterrupt: 

In [7]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from unidecode import unidecode
import hdbscan # Import the hdbscan library

def normalize_text(text):
    """Converts a string to its closest ASCII representation."""
    if isinstance(text, str):
        return unidecode(text)
    return text

def cluster_cities_with_hdbscan(
    gdf_geonames, 
    df_worldcities,
    min_cluster_size=5,
    min_samples=None
):
    """
    Identifies metropolitan areas by clustering cities using the HDBSCAN algorithm.
    This is often faster than OPTICS and requires less parameter tuning.

    Args:
        gdf_geonames (gpd.GeoDataFrame): Your primary GeoDataFrame of cities (your 'df').
                                         Must include columns: 'city_name', 'city_latitude',
                                         'city_longitude', 'city_population', 'geometry'.
        df_worldcities (pd.DataFrame): Your secondary DataFrame of major cities (your 'df2').
                                       Must include columns: 'city_ascii', 'lat',
                                       'lng', 'population', 'id'.
        min_samples (int): The number of samples in a neighborhood for a point to be
                           considered as a core point. This is the primary density parameter.
        xi (float): Determines the minimum steepness on the reachability plot that 
                    constitutes a cluster boundary. Between 0 and 1.

    Returns:
        gpd.GeoDataFrame: The original gdf_geonames with two new columns:
                          'metro_id' (the 'id' from df2 of the representative city) and
                          'metro_name' (the name of the representative city).
    """
    print("--- Starting HDBSCAN Clustering Process ---")

    # --- Step 1: Prepare and Combine Data ---
    print("Step 1: Preparing and standardizing data from both sources...")

    # Standardize GeoNames data (your 'df')
    gdf_geonames_std = gdf_geonames[['city_name', 'city_latitude', 'city_longitude', 'city_population', 'geometry']].copy()
    # Create a temporary unique ID from the index to merge back later
    gdf_geonames_std['original_id'] = gdf_geonames.index
    # Rename columns to a standard internal format
    gdf_geonames_std.rename(columns={
        'city_latitude': 'latitude',
        'city_longitude': 'longitude',
        'city_population': 'population'
    }, inplace=True)
    gdf_geonames_std['source'] = 'geonames'
    
    # Standardize and create GeoDataFrame for worldcities (your 'df2')
    df_worldcities_std = df_worldcities.copy()
    # Rename columns to the same standard internal format
    df_worldcities_std.rename(columns={
        'city_ascii': 'city_name',
        'lat': 'latitude',
        'lng': 'longitude',
        'id': 'original_id' # Use the worldcities ID as its unique identifier
    }, inplace=True)
    df_worldcities_std['source'] = 'worldcities'
    
    gdf_worldcities_std = gpd.GeoDataFrame(
        df_worldcities_std,
        geometry=gpd.points_from_xy(df_worldcities_std.longitude, df_worldcities_std.latitude),
        crs="EPSG:4326"
    )

    # Combine into a single GeoDataFrame using standardized columns
    cols_to_combine = ['original_id', 'city_name', 'latitude', 'longitude', 'population', 'geometry', 'source']
    gdf_all_cities = pd.concat([gdf_geonames_std[cols_to_combine], gdf_worldcities_std[cols_to_combine]], ignore_index=True)
    
    # Drop duplicates, keeping the one from worldcities if a conflict exists (as it's curated)
    gdf_all_cities.drop_duplicates(subset=['city_name', 'latitude', 'longitude'], keep='last', inplace=True)
    print(f"  Combined dataset has {len(gdf_all_cities)} unique cities.")

     # --- Step 2: Project Data and Run HDBSCAN ---
    print("\nStep 2: Projecting data and running HDBSCAN clustering...")
    print(f"  Parameters: min_cluster_size={min_cluster_size}")

    gdf_all_cities_proj = gdf_all_cities.to_crs(epsg=3857)
    coords = np.array(list(zip(gdf_all_cities_proj.geometry.x, gdf_all_cities_proj.geometry.y)))

    print("  Starting hdbscan.fit()...")

    # NOTE: HDBSCAN uses different parameters.
    # min_cluster_size is the most important one.
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=min_samples, # Often left as None to default to min_cluster_size
        metric='euclidean',
        # HDBSCAN has a prediction_data=True option for advanced use
    )
    clusterer.fit(coords)
        
    gdf_all_cities['cluster_id'] = clusterer.labels_
    num_clusters = len(np.unique(clusterer.labels_[clusterer.labels_ != -1]))
    num_noise = np.sum(clusterer.labels_ == -1)
    print(f"  HDBSCAN found {num_clusters} clusters and {num_noise} noise points.")

    # --- Step 3: Identify Representative City for Each Cluster ---
    print("\nStep 3: Identifying the most populous representative city for each cluster...")
    
    gdf_all_cities['population'] = pd.to_numeric(gdf_all_cities['population'], errors='coerce').fillna(0)
    
    # First, get the index labels of the most populous city for EACH cluster_id
    # This operates on the full dataframe.
    idx_of_representatives = gdf_all_cities.groupby('cluster_id')['population'].idxmax()

    # Now, use .loc on the full dataframe to select these rows
    representatives = gdf_all_cities.loc[idx_of_representatives].copy()

    # The result from idxmax() includes a row for the -1 "noise" cluster. We don't need it.
    representatives = representatives[representatives['cluster_id'] != -1]
    
    # Create a map from cluster_id to the representative's name and ID
    representatives_map_df = representatives[['cluster_id', 'city_name', 'original_id']].copy()
    representatives_map_df.rename(columns={'city_name': 'metro_name', 'original_id': 'metro_id'}, inplace=True)
    
    # --- Step 4: Map Representatives Back to Original GeoNames Data ---
    print("\nStep 4: Merging cluster results back into the original 'df' DataFrame...")
    
    # First, add the original_id to the main combined list so we can find it
    gdf_all_cities_with_id = gdf_all_cities.merge(gdf_geonames_std[['original_id', 'geometry']], on='geometry')

    # Merge the cluster information back to the main city list
    gdf_all_cities_merged = gdf_all_cities_with_id.merge(
        representatives_map_df, on='cluster_id', how='left'
    )
    
    # For noise points (not in a cluster), they represent themselves
    is_noise = gdf_all_cities_merged['metro_name'].isna()
    gdf_all_cities_merged.loc[is_noise, 'metro_name'] = gdf_all_cities_merged.loc[is_noise, 'city_name']
    gdf_all_cities_merged.loc[is_noise, 'metro_id'] = gdf_all_cities_merged.loc[is_noise, 'original_id_x'] # Use its own id
    
    # Filter for the original data source and the columns we need to add back
    final_cols_to_merge = gdf_all_cities_merged[gdf_all_cities_merged['source'] == 'geonames'][['original_id_x', 'metro_id', 'metro_name']]
    final_cols_to_merge.rename(columns={'original_id_x': 'original_id'}, inplace=True)
    final_cols_to_merge.drop_duplicates(subset=['original_id'], inplace=True)

    # Merge the new metro columns back to the original gdf_geonames using the index
    gdf_final = gdf_geonames.merge(
        final_cols_to_merge,
        left_index=True,
        right_on='original_id',
        how='left'
    )
    gdf_final.drop(columns=['original_id'], inplace=True, errors='ignore')

    print("\n--- HDBSCAN Process Complete! ---")
    print(f"Final dataset contains {len(gdf_final)} cities, enriched with metro area information.")
    
    return gdf_final

# --- Example Usage ---
# Assuming 'df' and 'df2' are pre-loaded with the specified column structures.

final_clustered_gdf = cluster_cities_with_hdbscan(
    gdf_geonames=df, 
    df_worldcities=df2
)

--- Starting HDBSCAN Clustering Process ---
Step 1: Preparing and standardizing data from both sources...
  Combined dataset has 263629 unique cities.

Step 2: Projecting data and running HDBSCAN clustering...
  Parameters: min_cluster_size=5
  Starting hdbscan.fit()...


c:\Users\larry\Desktop\Geoguessr ML Proj\Geolocation-Project\venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\larry\Desktop\Geoguessr ML Proj\Geolocation-Project\venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


  HDBSCAN found 7771 clusters and 94908 noise points.

Step 3: Identifying the most populous representative city for each cluster...

Step 4: Merging cluster results back into the original 'df' DataFrame...

--- HDBSCAN Process Complete! ---
Final dataset contains 215735 cities, enriched with metro area information.


In [8]:
# print("\nExample: Results for New York City Area")
# Check for a major city name that exists in your df2 data
nyc_area = final_clustered_gdf[final_clustered_gdf['metro_name'] == 'New York'] 
nyc_area.head(300)

,city_name,city_latitude,city_longitude,city_country_code,city_population,city_admin1_name,city_admin2_name,geometry,metro_id,metro_name


In [9]:
final_clustered_gdf.head(17)

,city_name,city_latitude,city_longitude,city_country_code,city_population,city_admin1_name,city_admin2_name,geometry,metro_id,metro_name
0.0,Vila,42.53176,1.56654,AD,1418,Encamp,,POINT (1.56654 42.53176),1.020829e+09,Andorra la Vella
1.0,Soldeu,42.57688,1.66769,AD,602,Canillo,,POINT (1.66769 42.57688),1.020829e+09,Andorra la Vella
2.0,Sispony,42.53368,1.51613,AD,833,La Massana,,POINT (1.51613 42.53368),1.020829e+09,Andorra la Vella
3.0,El Tarter,42.57952,1.65362,AD,1052,Canillo,,POINT (1.65362 42.57952),1.020829e+09,Andorra la Vella
4.0,Sant Julià de Lòria,42.46372,1.49129,AD,8022,Sant Julià de Loria,,POINT (1.49129 42.46372),1.020829e+09,Andorra la Vella
5.0,Pas de la Casa,42.54277,1.73361,AD,2363,Encamp,,POINT (1.73361 42.54277),1.020829e+09,Andorra la Vella
6.0,Ordino,42.55623,1.53319,AD,3066,Ordino,,POINT (1.53319 42.55623),1.020829e+09,Andorra la Vella
7.0,les Escaldes,42.50729,1.53414,AD,15853,Escaldes-Engordany,,POINT (1.53414 42.50729),1.020829e+09,Andorra la Vella
8.0,la Massana,42.54499,1.51483,AD,7211,La Massana,,POINT (1.51483 42.54499),1.020829e+09,Andorra la Vella
9.0,l'Aldosa,42.54391,1.52289,AD,594,La Massana,,POINT (1.52289 42.54391),1.020829e+09,Andorra la Vella


In [10]:
df[df["city_name"] == "Andorra la Vella"]

,city_name,city_latitude,city_longitude,city_country_code,city_population,city_admin1_name,city_admin2_name,geometry
14,Andorra la Vella,42.50779,1.52109,AD,20430,Andorra la Vella,,POINT (1.52109 42.50779)


In [11]:
df.head(30)

,city_name,city_latitude,city_longitude,city_country_code,city_population,city_admin1_name,city_admin2_name,geometry
0,Vila,42.53176,1.56654,AD,1418,Encamp,,POINT (1.56654 42.53176)
1,Soldeu,42.57688,1.66769,AD,602,Canillo,,POINT (1.66769 42.57688)
2,Sispony,42.53368,1.51613,AD,833,La Massana,,POINT (1.51613 42.53368)
3,El Tarter,42.57952,1.65362,AD,1052,Canillo,,POINT (1.65362 42.57952)
4,Sant Julià de Lòria,42.46372,1.49129,AD,8022,Sant Julià de Loria,,POINT (1.49129 42.46372)
5,Pas de la Casa,42.54277,1.73361,AD,2363,Encamp,,POINT (1.73361 42.54277)
6,Ordino,42.55623,1.53319,AD,3066,Ordino,,POINT (1.53319 42.55623)
7,les Escaldes,42.50729,1.53414,AD,15853,Escaldes-Engordany,,POINT (1.53414 42.50729)
8,la Massana,42.54499,1.51483,AD,7211,La Massana,,POINT (1.51483 42.54499)
9,l'Aldosa,42.54391,1.52289,AD,594,La Massana,,POINT (1.52289 42.54391)


In [12]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from unidecode import unidecode
import hdbscan # Import the hdbscan library

def normalize_text(text):
    """Converts a string to its closest ASCII representation."""
    if isinstance(text, str):
        return unidecode(text)
    return text

def cluster_cities_with_hdbscan(
    gdf_geonames, 
    df_worldcities,
    min_cluster_size=5,
    min_samples=None
):
    """
    Identifies metropolitan areas by clustering cities using HDBSCAN, but constrains
    the final "metro_name" by administrative boundaries (admin1/state).

    Args:
        gdf_geonames (gpd.GeoDataFrame): Your primary GeoDataFrame of cities ('df').
                                         Must include 'city_name', 'city_latitude',
                                         'city_longitude', 'city_population', 'geometry',
                                         and 'city_admin1_name'.
        df_worldcities (pd.DataFrame): Your secondary DataFrame of major cities ('df2').
                                       Must include 'city_ascii', 'lat', 'lng',
                                       'population', 'id', and 'admin_name'.

    Returns:
        gpd.GeoDataFrame: The original gdf_geonames with two new columns: 'metro_id' and
                          'metro_name', respecting admin1 boundaries.
    """
    print("--- Starting HDBSCAN Clustering Process with Admin1 Constraint ---")

    # --- Step 1: Prepare and Combine Data (Now including admin1 names) ---
    print("Step 1: Preparing and standardizing data from both sources...")

    # Standardize GeoNames data (your 'df')
    gdf_geonames_std = gdf_geonames[['city_name', 'city_latitude', 'city_longitude', 'city_population', 'city_admin1_name', 'geometry']].copy()
    gdf_geonames_std['original_id'] = gdf_geonames.index
    gdf_geonames_std.rename(columns={
        'city_latitude': 'latitude',
        'city_longitude': 'longitude',
        'city_population': 'population',
        'city_admin1_name': 'admin1_name' # Standardize admin1 column name
    }, inplace=True)
    gdf_geonames_std['source'] = 'geonames'
    
    # Standardize and create GeoDataFrame for worldcities (your 'df2')
    df_worldcities_std = df_worldcities.copy()
    df_worldcities_std.rename(columns={
        'city_ascii': 'city_name',
        'lat': 'latitude',
        'lng': 'longitude',
        'id': 'original_id',
        'admin_name': 'admin1_name' # Standardize admin1 column name
    }, inplace=True)
    df_worldcities_std['source'] = 'worldcities'
    
    gdf_worldcities_std = gpd.GeoDataFrame(
        df_worldcities_std,
        geometry=gpd.points_from_xy(df_worldcities_std.longitude, df_worldcities_std.latitude),
        crs="EPSG:4326"
    )

    # Combine into a single GeoDataFrame using standardized columns
    cols_to_combine = ['original_id', 'city_name', 'latitude', 'longitude', 'population', 'admin1_name', 'geometry', 'source']
    gdf_all_cities = pd.concat([gdf_geonames_std[cols_to_combine], gdf_worldcities_std[cols_to_combine]], ignore_index=True)
    
    gdf_all_cities.drop_duplicates(subset=['city_name', 'latitude', 'longitude'], keep='last', inplace=True)
    gdf_all_cities['admin1_name'] = gdf_all_cities['admin1_name'].apply(normalize_text) # Normalize admin names for clean grouping
    print(f"  Combined dataset has {len(gdf_all_cities)} unique cities.")

    # --- Step 2: Project Data and Run HDBSCAN (Unchanged) ---
    print("\nStep 2: Projecting data and running HDBSCAN clustering...")
    print(f"  Parameters: min_cluster_size={min_cluster_size}")

    gdf_all_cities_proj = gdf_all_cities.to_crs(epsg=3857)
    coords = np.array(list(zip(gdf_all_cities_proj.geometry.x, gdf_all_cities_proj.geometry.y)))

    print("  Starting hdbscan.fit()...")
    clusterer = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples, metric='euclidean')
    clusterer.fit(coords)
        
    gdf_all_cities['cluster_id'] = clusterer.labels_
    num_clusters = len(np.unique(clusterer.labels_[clusterer.labels_ != -1]))
    num_noise = np.sum(clusterer.labels_ == -1)
    print(f"  HDBSCAN found {num_clusters} clusters and {num_noise} noise points.")

    # --- Step 3: Identify Representative City for Each (Cluster + Admin1) Group ---
    print("\nStep 3: Identifying representative city for each [cluster, admin1] group...")
    
    gdf_all_cities['population'] = pd.to_numeric(gdf_all_cities['population'], errors='coerce').fillna(0)
    
    # ### --- KEY CHANGE --- ###
    # Group by BOTH cluster_id AND admin1_name to find the most populous city within each state for each cluster.
    idx_of_representatives = gdf_all_cities[gdf_all_cities['cluster_id'] != -1].groupby(['cluster_id', 'admin1_name'])['population'].idxmax()
    
    representatives = gdf_all_cities.loc[idx_of_representatives].copy()
    # ### --- END OF KEY CHANGE --- ###
    
    # Create a map from [cluster_id, admin1_name] to the representative's name and ID
    representatives_map_df = representatives[['cluster_id', 'admin1_name', 'city_name', 'original_id']].copy()
    representatives_map_df.rename(columns={'city_name': 'metro_name', 'original_id': 'metro_id'}, inplace=True)
    
    # --- Step 4: Map Representatives Back Using Both Cluster and Admin1 ---
    print("\nStep 4: Merging cluster results back into the original 'df' DataFrame...")
    
    # ### --- KEY CHANGE --- ###
    # Merge on both cluster_id and admin1_name. This ensures a city in NJ gets the NJ representative.
    gdf_all_cities_merged = gdf_all_cities.merge(
        representatives_map_df,
        on=['cluster_id', 'admin1_name'], 
        how='left'
    )
    # ### --- END OF KEY CHANGE --- ###
    
    # For noise points or cities in a cluster but a different admin1 (now handled by the merge), they represent themselves
    is_noise_or_unmatched = gdf_all_cities_merged['metro_name'].isna()
    gdf_all_cities_merged.loc[is_noise_or_unmatched, 'metro_name'] = gdf_all_cities_merged.loc[is_noise_or_unmatched, 'city_name']
    gdf_all_cities_merged.loc[is_noise_or_unmatched, 'metro_id'] = gdf_all_cities_merged.loc[is_noise_or_unmatched, 'original_id']
    
    # Filter for the original data source and the columns we need to add back
    final_cols_to_merge = gdf_all_cities_merged[gdf_all_cities_merged['source'] == 'geonames'][['original_id', 'metro_id', 'metro_name']]
    final_cols_to_merge.drop_duplicates(subset=['original_id'], inplace=True)

    # Merge the new metro columns back to the original gdf_geonames using the index
    gdf_final = gdf_geonames.merge(
        final_cols_to_merge,
        left_index=True,
        right_on='original_id',
        how='left'
    )
    gdf_final.drop(columns=['original_id'], inplace=True, errors='ignore')

    print("\n--- Process Complete! ---")
    print(f"Final dataset contains {len(gdf_final)} cities, enriched with metro area information.")
    
    return gdf_final
final_clustered_gdf2 = cluster_cities_with_hdbscan(
    gdf_geonames=df, 
    df_worldcities=df2
)

--- Starting HDBSCAN Clustering Process with Admin1 Constraint ---
Step 1: Preparing and standardizing data from both sources...
  Combined dataset has 263629 unique cities.

Step 2: Projecting data and running HDBSCAN clustering...
  Parameters: min_cluster_size=5
  Starting hdbscan.fit()...


c:\Users\larry\Desktop\Geoguessr ML Proj\Geolocation-Project\venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\larry\Desktop\Geoguessr ML Proj\Geolocation-Project\venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


  HDBSCAN found 7771 clusters and 94908 noise points.

Step 3: Identifying representative city for each [cluster, admin1] group...

Step 4: Merging cluster results back into the original 'df' DataFrame...

--- Process Complete! ---
Final dataset contains 215735 cities, enriched with metro area information.


In [ ]:
# print("\nExample: Results for New York City Area")
# Check for a major city name that exists in your df2 data
nyc_area = final_clustered_gdf2[final_clustered_gdf2['metro_name'] == 'New York'] 
nyc_area.head(3)

,city_name,city_latitude,city_longitude,city_country_code,city_population,city_admin1_name,city_admin2_name,geometry,metro_id,metro_name


In [14]:
final_clustered_gdf2.head(17)

,city_name,city_latitude,city_longitude,city_country_code,city_population,city_admin1_name,city_admin2_name,geometry,metro_id,metro_name
0.0,Vila,42.53176,1.56654,AD,1418,Encamp,,POINT (1.56654 42.53176),1.020417e+09,Encamp
1.0,Soldeu,42.57688,1.66769,AD,602,Canillo,,POINT (1.66769 42.57688),1.020983e+09,Sant Pere
2.0,Sispony,42.53368,1.51613,AD,833,La Massana,,POINT (1.51613 42.53368),1.020543e+09,La Massana
3.0,El Tarter,42.57952,1.65362,AD,1052,Canillo,,POINT (1.65362 42.57952),1.020983e+09,Sant Pere
4.0,Sant Julià de Lòria,42.46372,1.49129,AD,8022,Sant Julià de Loria,,POINT (1.49129 42.46372),1.020886e+09,Sant Julia de Loria
5.0,Pas de la Casa,42.54277,1.73361,AD,2363,Encamp,,POINT (1.73361 42.54277),1.020417e+09,Encamp
6.0,Ordino,42.55623,1.53319,AD,3066,Ordino,,POINT (1.53319 42.55623),1.020655e+09,Ordino
7.0,les Escaldes,42.50729,1.53414,AD,15853,Escaldes-Engordany,,POINT (1.53414 42.50729),7.000000e+00,les Escaldes
8.0,la Massana,42.54499,1.51483,AD,7211,La Massana,,POINT (1.51483 42.54499),1.020543e+09,La Massana
9.0,l'Aldosa,42.54391,1.52289,AD,594,La Massana,,POINT (1.52289 42.54391),1.020543e+09,La Massana


In [15]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from unidecode import unidecode
import hdbscan

def normalize_text(text):
    """Converts a string to its closest ASCII representation."""
    if isinstance(text, str):
        return unidecode(text)
    return text

def cluster_cities_with_hdbscan_two_tier(
    gdf_geonames, 
    df_worldcities,
    min_cluster_size=5,
    min_samples=None
):
    """
    Identifies both local and greater metropolitan areas using a two-tiered approach.
    - Tier 1 (greater_metro_name): Purely spatial clustering to find major metropolitan areas (e.g., "New York City").
    - Tier 2 (metro_name): Constrained by admin1/admin2 to find local hubs (e.g., "Brooklyn", "Jersey City").

    Args:
        gdf_geonames (gpd.GeoDataFrame): Your primary GeoDataFrame ('df'). Requires columns:
                                         'city_name', 'city_latitude', 'city_longitude', 
                                         'city_population', 'city_admin1_name', 'city_admin2_name'.
        df_worldcities (pd.DataFrame): Your secondary DataFrame ('df2'). Requires columns:
                                       'city_ascii', 'lat', 'lng', 'population', 'id', 
                                       'admin_name'. 'admin2' data not used from here.

    Returns:
        gpd.GeoDataFrame: The original gdf_geonames with two new columns:
                          'metro_name', 'metro_id', 'greater_metro_name', 'greater_metro_id'.
    """
    print("--- Starting Two-Tier HDBSCAN Clustering Process ---")

    # --- Step 1: Prepare and Combine Data (Now including admin2 names) ---
    print("Step 1: Preparing and standardizing data from both sources...")

    # Standardize GeoNames data ('df')
    cols_needed_geonames = ['city_name', 'city_latitude', 'city_longitude', 'city_population', 'city_admin1_name', 'city_admin2_name', 'geometry']
    gdf_geonames_std = gdf_geonames[cols_needed_geonames].copy()
    gdf_geonames_std['original_id'] = gdf_geonames.index
    gdf_geonames_std.rename(columns={
        'city_latitude': 'latitude', 'city_longitude': 'longitude',
        'city_population': 'population', 'city_admin1_name': 'admin1_name',
        'city_admin2_name': 'admin2_name'
    }, inplace=True)
    gdf_geonames_std['source'] = 'geonames'

    # Standardize worldcities data ('df2')
    df_worldcities_std = df_worldcities.copy()
    df_worldcities_std.rename(columns={
        'city_ascii': 'city_name', 'lat': 'latitude', 'lng': 'longitude',
        'id': 'original_id', 'admin_name': 'admin1_name'
    }, inplace=True)
    df_worldcities_std['source'] = 'worldcities'
    # Fill admin2 with a placeholder as it's not present in df2
    df_worldcities_std['admin2_name'] = pd.NA 
    
    gdf_worldcities_std = gpd.GeoDataFrame(
        df_worldcities_std,
        geometry=gpd.points_from_xy(df_worldcities_std.longitude, df_worldcities_std.latitude),
        crs="EPSG:4326"
    )

    # Combine into a single GeoDataFrame
    cols_to_combine = ['original_id', 'city_name', 'latitude', 'longitude', 'population', 'admin1_name', 'admin2_name', 'geometry', 'source']
    gdf_all_cities = pd.concat([gdf_geonames_std[cols_to_combine], gdf_worldcities_std[cols_to_combine]], ignore_index=True)
    
    gdf_all_cities.drop_duplicates(subset=['city_name', 'latitude', 'longitude'], keep='last', inplace=True)
    
    # Normalize text and handle missing admin names for clean grouping
    for col in ['admin1_name', 'admin2_name']:
        gdf_all_cities[col] = gdf_all_cities[col].apply(normalize_text).fillna('__NONE__')

    print(f"  Combined dataset has {len(gdf_all_cities)} unique cities.")

    # --- Step 2: Project Data and Run HDBSCAN (Unchanged) ---
    print("\nStep 2: Running HDBSCAN clustering...")
    gdf_all_cities_proj = gdf_all_cities.to_crs(epsg=3857)
    coords = np.array(list(zip(gdf_all_cities_proj.geometry.x, gdf_all_cities_proj.geometry.y)))
    clusterer = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples, metric='euclidean')
    clusterer.fit(coords)
    gdf_all_cities['cluster_id'] = clusterer.labels_
    print(f"  HDBSCAN found {len(np.unique(clusterer.labels_)) - 1} clusters.")

    # --- Step 3: Identify Representatives for Both Tiers ---
    print("\nStep 3: Identifying representatives for both local and greater metro areas...")
    gdf_all_cities['population'] = pd.to_numeric(gdf_all_cities['population'], errors='coerce').fillna(0)

    # Tier 1: Greater Metro Area (most populous in the whole cluster)
    idx_greater = gdf_all_cities.groupby('cluster_id')['population'].idxmax()
    reps_greater = gdf_all_cities.loc[idx_greater].copy()
    reps_greater = reps_greater[reps_greater['cluster_id'] != -1]
    greater_map = reps_greater[['cluster_id', 'city_name', 'original_id']].rename(
        columns={'city_name': 'greater_metro_name', 'original_id': 'greater_metro_id'}
    )

    # Tier 2: Local Metro Hub (most populous within cluster/admin1/admin2)
    idx_local = gdf_all_cities[gdf_all_cities['cluster_id'] != -1].groupby(['cluster_id', 'admin1_name', 'admin2_name'])['population'].idxmax()
    reps_local = gdf_all_cities.loc[idx_local].copy()
    local_map = reps_local[['cluster_id', 'admin1_name', 'admin2_name', 'city_name', 'original_id']].rename(
        columns={'city_name': 'metro_name', 'original_id': 'metro_id'}
    )

    # --- Step 4: Map Both Tiers Back to the Main DataFrame ---
    print("\nStep 4: Merging two-tier results back into the original 'df' DataFrame...")

    # Merge Greater Metro Name (on cluster_id)
    gdf_all_merged = gdf_all_cities.merge(greater_map, on='cluster_id', how='left')
    
    # Merge Local Metro Name (on cluster_id, admin1, admin2)
    gdf_all_merged = gdf_all_merged.merge(local_map, on=['cluster_id', 'admin1_name', 'admin2_name'], how='left')

    # For noise points or unmatched cities, they represent themselves in both tiers
    for tier in ['metro', 'greater_metro']:
        is_unmatched = gdf_all_merged[f'{tier}_name'].isna()
        gdf_all_merged.loc[is_unmatched, f'{tier}_name'] = gdf_all_merged.loc[is_unmatched, 'city_name']
        gdf_all_merged.loc[is_unmatched, f'{tier}_id'] = gdf_all_merged.loc[is_unmatched, 'original_id']
    
    # Filter for the original data source and merge back
    cols_to_merge = ['original_id', 'metro_name', 'metro_id', 'greater_metro_name', 'greater_metro_id']
    final_cols_to_merge = gdf_all_merged[gdf_all_merged['source'] == 'geonames'][cols_to_merge]
    final_cols_to_merge.drop_duplicates(subset=['original_id'], inplace=True)

    gdf_final = gdf_geonames.merge(
        final_cols_to_merge,
        left_index=True,
        right_on='original_id',
        how='left'
    )
    gdf_final.drop(columns=['original_id'], inplace=True, errors='ignore')

    print("\n--- Process Complete! ---")
    return gdf_final
final_clustered_gdf3 = cluster_cities_with_hdbscan_two_tier(
    gdf_geonames=df, 
    df_worldcities=df2
)

--- Starting Two-Tier HDBSCAN Clustering Process ---
Step 1: Preparing and standardizing data from both sources...
  Combined dataset has 263629 unique cities.

Step 2: Running HDBSCAN clustering...


c:\Users\larry\Desktop\Geoguessr ML Proj\Geolocation-Project\venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\larry\Desktop\Geoguessr ML Proj\Geolocation-Project\venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


  HDBSCAN found 7771 clusters.

Step 3: Identifying representatives for both local and greater metro areas...

Step 4: Merging two-tier results back into the original 'df' DataFrame...

--- Process Complete! ---


In [31]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from unidecode import unidecode
import hdbscan

def normalize_text(text):
    # (function remains the same)
    if isinstance(text, str):
        return unidecode(text)
    return text

def cluster_cities_with_hdbscan_two_tier_tune(
    gdf_geonames, 
    df_worldcities,
    min_cluster_size=15, # Increased default for tighter clusters
    min_samples=15,      # Increased default for tighter clusters
    cluster_selection_epsilon_meters=None # Optional: hard distance limit
):
    """
    Identifies metropolitan areas with tunable cluster sizes.

    Args:
        gdf_geonames (gpd.GeoDataFrame): Your primary city DataFrame ('df').
        df_worldcities (pd.DataFrame): Your secondary major city DataFrame ('df2').
        min_cluster_size (int): The minimum number of cities required to form a cluster.
                                HIGHER values lead to TIGHTER, more urban clusters.
        min_samples (int): Controls how conservative clustering is. Higher values make
                           clusters denser. Often set to the same as min_cluster_size.
                           Set to None to have it default to min_cluster_size.
        cluster_selection_epsilon_meters (float, optional): A hard distance limit. No cluster
                                                            can span a connection larger than this
                                                            distance in meters. e.g., 50000 for 50km.
                                                            Defaults to None (no limit).
    """
    print("--- Starting Two-Tier HDBSCAN Clustering Process ---")

    # --- Step 1: Prepare and Combine Data (Unchanged) ---
    # ... (code for this step is identical to the previous version)
    print("Step 1: Preparing and standardizing data...")
    cols_needed_geonames = ['city_name', 'city_latitude', 'city_longitude', 'city_population', 'city_admin1_name', 'city_admin2_name', 'geometry']
    gdf_geonames_std = gdf_geonames[cols_needed_geonames].copy()
    gdf_geonames_std['original_id'] = gdf_geonames.index
    gdf_geonames_std.rename(columns={
        'city_latitude': 'latitude', 'city_longitude': 'longitude',
        'city_population': 'population', 'city_admin1_name': 'admin1_name',
        'city_admin2_name': 'admin2_name'
    }, inplace=True)
    gdf_geonames_std['source'] = 'geonames'

    df_worldcities_std = df_worldcities.copy()
    df_worldcities_std.rename(columns={
        'city_ascii': 'city_name', 'lat': 'latitude', 'lng': 'longitude',
        'id': 'original_id', 'admin_name': 'admin1_name'
    }, inplace=True)
    df_worldcities_std['source'] = 'worldcities'
    df_worldcities_std['admin2_name'] = pd.NA 
    
    gdf_worldcities_std = gpd.GeoDataFrame(
        df_worldcities_std,
        geometry=gpd.points_from_xy(df_worldcities_std.longitude, df_worldcities_std.latitude),
        crs="EPSG:4326"
    )

    cols_to_combine = ['original_id', 'city_name', 'latitude', 'longitude', 'population', 'admin1_name', 'admin2_name', 'geometry', 'source']
    gdf_all_cities = pd.concat([gdf_geonames_std[cols_to_combine], gdf_worldcities_std[cols_to_combine]], ignore_index=True)
    gdf_all_cities.drop_duplicates(subset=['city_name', 'latitude', 'longitude'], keep='last', inplace=True)
    for col in ['admin1_name', 'admin2_name']:
        gdf_all_cities[col] = gdf_all_cities[col].apply(normalize_text).fillna('__NONE__')
    print(f"  Combined dataset has {len(gdf_all_cities)} unique cities.")


    # --- Step 2: Project Data and Run HDBSCAN with New Parameters ---
    print("\nStep 2: Running HDBSCAN clustering...")
    print(f"  Tuning Parameters: min_cluster_size={min_cluster_size}, min_samples={min_samples}, epsilon={cluster_selection_epsilon_meters}m")

    gdf_all_cities_proj = gdf_all_cities.to_crs(epsg=3857)
    coords = np.array(list(zip(gdf_all_cities_proj.geometry.x, gdf_all_cities_proj.geometry.y)))

    # ### --- KEY CHANGE --- ###
    # Pass the new tuning parameters to the HDBSCAN constructor
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=min_samples,
        metric='euclidean',
        cluster_selection_epsilon=cluster_selection_epsilon_meters or 0 # Epsilon must be a number
    )
    # ### --- END OF KEY CHANGE --- ###

    clusterer.fit(coords)
    gdf_all_cities['cluster_id'] = clusterer.labels_
    print(f"  HDBSCAN found {len(np.unique(clusterer.labels_)) - 1} clusters.")

    # --- Step 3 & 4: Identifying and Merging (Unchanged) ---
    # ... (code for these steps is identical to the previous version)
    print("\nStep 3: Identifying representatives for both local and greater metro areas...")
    gdf_all_cities['population'] = pd.to_numeric(gdf_all_cities['population'], errors='coerce').fillna(0)
    idx_greater = gdf_all_cities.groupby('cluster_id')['population'].idxmax()
    reps_greater = gdf_all_cities.loc[idx_greater].copy()
    reps_greater = reps_greater[reps_greater['cluster_id'] != -1]
    greater_map = reps_greater[['cluster_id', 'city_name', 'original_id']].rename(
        columns={'city_name': 'greater_metro_name', 'original_id': 'greater_metro_id'}
    )
    idx_local = gdf_all_cities[gdf_all_cities['cluster_id'] != -1].groupby(['cluster_id', 'admin1_name', 'admin2_name'])['population'].idxmax()
    reps_local = gdf_all_cities.loc[idx_local].copy()
    local_map = reps_local[['cluster_id', 'admin1_name', 'admin2_name', 'city_name', 'original_id']].rename(
        columns={'city_name': 'metro_name', 'original_id': 'metro_id'}
    )
    print("\nStep 4: Merging two-tier results back into the original 'df' DataFrame...")
    gdf_all_merged = gdf_all_cities.merge(greater_map, on='cluster_id', how='left')
    gdf_all_merged = gdf_all_merged.merge(local_map, on=['cluster_id', 'admin1_name', 'admin2_name'], how='left')
    for tier in ['metro', 'greater_metro']:
        is_unmatched = gdf_all_merged[f'{tier}_name'].isna()
        gdf_all_merged.loc[is_unmatched, f'{tier}_name'] = gdf_all_merged.loc[is_unmatched, 'city_name']
        gdf_all_merged.loc[is_unmatched, f'{tier}_id'] = gdf_all_merged.loc[is_unmatched, 'original_id']
    
    cols_to_merge = ['original_id', 'metro_name', 'metro_id', 'greater_metro_name', 'greater_metro_id']
    final_cols_to_merge = gdf_all_merged[gdf_all_merged['source'] == 'geonames'][cols_to_merge]
    final_cols_to_merge.drop_duplicates(subset=['original_id'], inplace=True)

    gdf_final = gdf_geonames.merge(
        final_cols_to_merge,
        left_index=True,
        right_on='original_id',
        how='left'
    )
    gdf_final.drop(columns=['original_id'], inplace=True, errors='ignore')

    print("\n--- Process Complete! ---")
    return gdf_final

final_gdf_tuned = cluster_cities_with_hdbscan_two_tier_tune(
    df, 
    df2,
    min_cluster_size=5, # A much higher requirement
    min_samples=25       # Keep this matched for now
)

--- Starting Two-Tier HDBSCAN Clustering Process ---
Step 1: Preparing and standardizing data...
  Combined dataset has 263629 unique cities.

Step 2: Running HDBSCAN clustering...
  Tuning Parameters: min_cluster_size=5, min_samples=25, epsilon=Nonem


c:\Users\larry\Desktop\Geoguessr ML Proj\Geolocation-Project\venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\larry\Desktop\Geoguessr ML Proj\Geolocation-Project\venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


  HDBSCAN found 1400 clusters.

Step 3: Identifying representatives for both local and greater metro areas...

Step 4: Merging two-tier results back into the original 'df' DataFrame...

--- Process Complete! ---


In [ ]:
final_clustered_gdf3[(final_clustered_gdf3["greater_metro_name"] == "Phoenix") & (final_clustered_gdf3["city_admin1_name"] == "Arizona")]

In [ ]:
final_gdf_tuned[(final_gdf_tuned["metro_name"] == "El Paso") & (final_gdf_tuned["city_admin1_name"] == "Texas")]

In [ ]:
final_gdf_tuned[(final_gdf_tuned["greater_metro_name"] == "New York City")]

In [ ]:
final_clustered_gdf3[(final_clustered_gdf3["greater_metro_name"] == "New York City")]

In [53]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from unidecode import unidecode
import hdbscan

def normalize_text(text):
    """Converts a string to its closest ASCII representation."""
    if isinstance(text, str):
        return unidecode(text)
    return text

def cluster_cities_with_hdbscan_three_tier(
    gdf_geonames, 
    df_worldcities,
    min_cluster_size=5,
    min_samples=None,
    cluster_selection_epsilon_meters=None
):
    """
    Identifies a three-tiered metropolitan hierarchy using HDBSCAN.

    Args:
        gdf_geonames (gpd.GeoDataFrame): Your primary city DataFrame ('df'). Requires:
                                         'city_name', 'city_latitude', 'city_longitude', 
                                         'city_population', 'city_country_code', 'city_admin1_name', 
                                         'city_admin2_name'.
        df_worldcities (pd.DataFrame): Your secondary major city DataFrame ('df2'). Requires:
                                       'city_ascii', 'lat', 'lng', 'population', 'id', 
                                       'iso2', 'admin_name'.
        min_cluster_size, min_samples, cluster_selection_epsilon_meters: Tuning parameters for HDBSCAN.

    Returns:
        gpd.GeoDataFrame: The original gdf_geonames with six new columns representing three tiers of metro areas:
                          - metro_name/id: Most granular, respects county/state/country.
                          - greater_metro_in_country_name/id: Broad metro area, but respects country borders.
                          - greater_metro_name/id: True MSA, can cross all borders.
    """
    print("--- Starting Three-Tier HDBSCAN Clustering Process ---")

    # --- Step 1: Prepare and Combine Data (Now including country codes) ---
    print("Step 1: Preparing and standardizing data...")
    
    # Standardize GeoNames data ('df')
    cols_needed_geonames = ['city_name', 'city_latitude', 'city_longitude', 'city_population', 'city_country_code', 'city_admin1_name', 'city_admin2_name', 'geometry']
    gdf_geonames_std = gdf_geonames[cols_needed_geonames].copy()
    gdf_geonames_std['original_id'] = gdf_geonames.index
    gdf_geonames_std.rename(columns={
        'city_latitude': 'latitude', 'city_longitude': 'longitude',
        'city_population': 'population', 'city_country_code': 'country_code',
        'city_admin1_name': 'admin1_name', 'city_admin2_name': 'admin2_name'
    }, inplace=True)
    gdf_geonames_std['source'] = 'geonames'

    # Standardize worldcities data ('df2')
    df_worldcities_std = df_worldcities.copy()
    df_worldcities_std.rename(columns={
        'city_ascii': 'city_name', 'lat': 'latitude', 'lng': 'longitude',
        'id': 'original_id', 'iso2': 'country_code', 'admin_name': 'admin1_name'
    }, inplace=True)
    df_worldcities_std['source'] = 'worldcities'
    df_worldcities_std['admin2_name'] = pd.NA
    
    gdf_worldcities_std = gpd.GeoDataFrame(
        df_worldcities_std,
        geometry=gpd.points_from_xy(df_worldcities_std.longitude, df_worldcities_std.latitude),
        crs="EPSG:4326"
    )

    # Combine into a single GeoDataFrame
    cols_to_combine = ['original_id', 'city_name', 'latitude', 'longitude', 'population', 'country_code', 'admin1_name', 'admin2_name', 'geometry', 'source']
    gdf_all_cities = pd.concat([gdf_geonames_std[cols_to_combine], gdf_worldcities_std[cols_to_combine]], ignore_index=True)
    
    gdf_all_cities.drop_duplicates(subset=['city_name', 'latitude', 'longitude'], keep='last', inplace=True)
    for col in ['admin1_name', 'admin2_name']:
        gdf_all_cities[col] = gdf_all_cities[col].apply(normalize_text).fillna('__NONE__')
    print(f"  Combined dataset has {len(gdf_all_cities)} unique cities.")

    # --- Step 2: Project Data and Run HDBSCAN (Unchanged) ---
    print("\nStep 2: Running HDBSCAN clustering...")
    gdf_all_cities_proj = gdf_all_cities.to_crs(epsg=3857)
    coords = np.array(list(zip(gdf_all_cities_proj.geometry.x, gdf_all_cities_proj.geometry.y)))
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size, min_samples=min_samples, metric='euclidean',
        cluster_selection_epsilon=cluster_selection_epsilon_meters or 0
    )
    clusterer.fit(coords)
    gdf_all_cities['cluster_id'] = clusterer.labels_
    print(f"  HDBSCAN found {len(np.unique(clusterer.labels_)) - 1} clusters.")

    # --- Step 3: Identify Representatives for All Three Tiers ---
    print("\nStep 3: Identifying representatives for all three metro tiers...")
    gdf_all_cities['population'] = pd.to_numeric(gdf_all_cities['population'], errors='coerce').fillna(0)
    clustered_cities = gdf_all_cities[gdf_all_cities['cluster_id'] != -1]

    # Tier 1: greater_metro (purely spatial)
    idx_greater = clustered_cities.groupby('cluster_id')['population'].idxmax()
    reps_greater = gdf_all_cities.loc[idx_greater].copy()
    greater_map = reps_greater[['cluster_id', 'city_name', 'original_id']].rename(
        columns={'city_name': 'greater_metro_name', 'original_id': 'greater_metro_id'}
    )
    
    # Tier 2: greater_metro_in_country (spatial, but respects country borders)
    idx_greater_country = clustered_cities.groupby(['cluster_id', 'country_code'])['population'].idxmax()
    reps_greater_country = gdf_all_cities.loc[idx_greater_country].copy()
    greater_country_map = reps_greater_country[['cluster_id', 'country_code', 'city_name', 'original_id']].rename(
        columns={'city_name': 'greater_metro_in_country_name', 'original_id': 'greater_metro_in_country_id'}
    )

    # Tier 3: metro (most granular, respects all admin boundaries)
    idx_local = clustered_cities.groupby(['cluster_id', 'country_code', 'admin1_name', 'admin2_name'])['population'].idxmax()
    reps_local = gdf_all_cities.loc[idx_local].copy()
    local_map = reps_local[['cluster_id', 'country_code', 'admin1_name', 'admin2_name', 'city_name', 'original_id']].rename(
        columns={'city_name': 'metro_name', 'original_id': 'metro_id'}
    )

    # --- Step 4: Map All Tiers Back to the Main DataFrame ---
    print("\nStep 4: Merging three-tier results back into the original 'df' DataFrame...")

    gdf_all_merged = gdf_all_cities.merge(greater_map, on='cluster_id', how='left')
    gdf_all_merged = gdf_all_merged.merge(greater_country_map, on=['cluster_id', 'country_code'], how='left')
    gdf_all_merged = gdf_all_merged.merge(local_map, on=['cluster_id', 'country_code', 'admin1_name', 'admin2_name'], how='left')

    # For noise/unmatched points, they represent themselves in all tiers
    for tier_prefix in ['metro', 'greater_metro_in_country', 'greater_metro']:
        is_unmatched = gdf_all_merged[f'{tier_prefix}_name'].isna()
        gdf_all_merged.loc[is_unmatched, f'{tier_prefix}_name'] = gdf_all_merged.loc[is_unmatched, 'city_name']
        gdf_all_merged.loc[is_unmatched, f'{tier_prefix}_id'] = gdf_all_merged.loc[is_unmatched, 'original_id']
    
    # Filter for the original data source and merge back
    cols_to_merge = [
        'original_id', 'metro_name', 'metro_id', 'greater_metro_in_country_name', 
        'greater_metro_in_country_id', 'greater_metro_name', 'greater_metro_id', 'source'
    ]
    final_cols_to_merge = gdf_all_merged[gdf_all_merged['source'] == 'geonames'][cols_to_merge]
    final_cols_to_merge.drop_duplicates(subset=['original_id'], inplace=True)

    gdf_final = gdf_geonames.merge(
        final_cols_to_merge,
        left_index=True,
        right_on='original_id',
        how='left'
    )
    gdf_final.drop(columns=['original_id'], inplace=True, errors='ignore')

    print("\n--- Process Complete! ---")
    return gdf_final

final_gdf_tuned = cluster_cities_with_hdbscan_three_tier(
    df, 
    df2
)

--- Starting Three-Tier HDBSCAN Clustering Process ---
Step 1: Preparing and standardizing data...
  Combined dataset has 263629 unique cities.

Step 2: Running HDBSCAN clustering...


c:\Users\larry\Desktop\Geoguessr ML Proj\Geolocation-Project\venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\larry\Desktop\Geoguessr ML Proj\Geolocation-Project\venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


  HDBSCAN found 7771 clusters.

Step 3: Identifying representatives for all three metro tiers...

Step 4: Merging three-tier results back into the original 'df' DataFrame...

--- Process Complete! ---


In [41]:
final_gdf_tuned[(final_gdf_tuned["greater_metro_name"] == "Ciudad Juárez")]

,city_name,city_latitude,city_longitude,city_country_code,city_population,city_admin1_name,city_admin2_name,geometry,metro_name,metro_id,greater_metro_in_country_name,greater_metro_in_country_id,greater_metro_name,greater_metro_id
137110.0,Praxédis Guerrero,31.36667,-106.01667,MX,3787,Chihuahua,Praxedis G. Guerrero,POINT (-106.01667 31.36667),Praxédis Guerrero,137166.0,Ciudad Juárez,1.399450e+05,Ciudad Juárez,139945.0
137123.0,Jesús Carranza (La Colorada),31.49241,-106.23569,MX,509,Chihuahua,Juárez,POINT (-106.23569 31.49241),Ciudad Juárez,139945.0,Ciudad Juárez,1.399450e+05,Ciudad Juárez,139945.0
137133.0,Barreales,31.39806,-106.13917,MX,540,Chihuahua,Guadalupe,POINT (-106.13917 31.39806),Guadalupe,139365.0,Ciudad Juárez,1.399450e+05,Ciudad Juárez,139945.0
138075.0,San Isidro,31.54666,-106.28124,MX,3483,Chihuahua,Juárez,POINT (-106.28124 31.54666),Ciudad Juárez,139945.0,Ciudad Juárez,1.399450e+05,Ciudad Juárez,139945.0
138205.0,San Agustín,31.51674,-106.25548,MX,1359,Chihuahua,Juárez,POINT (-106.25548 31.51674),Ciudad Juárez,139945.0,Ciudad Juárez,1.399450e+05,Ciudad Juárez,139945.0
138211.0,Samalayuca,31.34242,-106.47981,MX,1474,Chihuahua,Juárez,POINT (-106.47981 31.34242),Ciudad Juárez,139945.0,Ciudad Juárez,1.399450e+05,Ciudad Juárez,139945.0
138347.0,Praxedis G. Guerrero,31.37061,-106.00616,MX,2128,Chihuahua,Praxedis G. Guerrero,POINT (-106.00616 31.37061),Praxédis Guerrero,137166.0,Ciudad Juárez,1.399450e+05,Ciudad Juárez,139945.0
138798.0,Loma Blanca,31.57972,-106.29860,MX,2169,Chihuahua,Juárez,POINT (-106.2986 31.57972),Ciudad Juárez,139945.0,Ciudad Juárez,1.399450e+05,Ciudad Juárez,139945.0
138909.0,Rinconada del Mimbre,31.34644,-106.04827,MX,539,Chihuahua,Guadalupe,POINT (-106.04827 31.34644),Guadalupe,139365.0,Ciudad Juárez,1.399450e+05,Ciudad Juárez,139945.0
139141.0,Juárez y Reforma,31.44000,-106.17056,MX,788,Chihuahua,Guadalupe,POINT (-106.17056 31.44),Guadalupe,139365.0,Ciudad Juárez,1.399450e+05,Ciudad Juárez,139945.0


In [54]:
final_gdf_tuned[final_gdf_tuned["city_name"].isnull()]

,city_name,city_latitude,city_longitude,city_country_code,city_population,city_admin1_name,city_admin2_name,geometry,metro_name,metro_id,greater_metro_in_country_name,greater_metro_in_country_id,greater_metro_name,greater_metro_id,source
115567.0,NaN,44.93645,7.54015,IT,7507,Piedmont,Torino,POINT (7.54015 44.93645),NaN,115608.0,NaN,115608.0,NaN,115608.0,geonames


In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from unidecode import unidecode
import hdbscan

def normalize_text(text):
    """Converts a string to its closest ASCII representation."""
    if isinstance(text, str):
        return unidecode(text)
    return text

def cluster_cities_with_hdbscan_three_tier(
    gdf_geonames, 
    df_worldcities,
    min_cluster_size=5,
    min_samples=None,
    cluster_selection_epsilon_meters=None
):
    """
    Identifies a three-tiered metropolitan hierarchy using HDBSCAN.

    Args:
        gdf_geonames (gpd.GeoDataFrame): Your primary city DataFrame ('df'). Requires:
                                         'city_name', 'city_latitude', 'city_longitude', 
                                         'city_population', 'city_country_code', 'city_admin1_name', 
                                         'city_admin2_name'.
        df_worldcities (pd.DataFrame): Your secondary major city DataFrame ('df2'). Requires:
                                       'city_ascii', 'lat', 'lng', 'population', 'id', 
                                       'iso2', 'admin_name'.
        min_cluster_size, min_samples, cluster_selection_epsilon_meters: Tuning parameters for HDBSCAN.

    Returns:
        gpd.GeoDataFrame: The original gdf_geonames with six new columns representing three tiers of metro areas:
                          - metro_name/id: Most granular, respects county/state/country.
                          - greater_metro_in_country_name/id: Broad metro area, but respects country borders.
                          - greater_metro_name/id: True MSA, can cross all borders.
    """
    print("--- Starting Three-Tier HDBSCAN Clustering Process ---")

    # --- Step 1: Prepare and Combine Data (Now including country codes) ---
    print("Step 1: Preparing and standardizing data...")
    
    # Standardize GeoNames data ('df')
    cols_needed_geonames = ['city_name', 'city_latitude', 'city_longitude', 'city_population', 'city_country_code', 'city_admin1_name', 'city_admin2_name', 'geometry']
    gdf_geonames_std = gdf_geonames[cols_needed_geonames].copy()
    gdf_geonames_std['original_id'] = gdf_geonames.index
    gdf_geonames_std.rename(columns={
        'city_latitude': 'latitude', 'city_longitude': 'longitude',
        'city_population': 'population', 'city_country_code': 'country_code',
        'city_admin1_name': 'admin1_name', 'city_admin2_name': 'admin2_name'
    }, inplace=True)
    gdf_geonames_std['source'] = 'geonames'

    # Standardize worldcities data ('df2')
    df_worldcities_std = df_worldcities.copy()
    df_worldcities_std.rename(columns={
        'city_ascii': 'city_name', 'lat': 'latitude', 'lng': 'longitude',
        'id': 'original_id', 'iso2': 'country_code', 'admin_name': 'admin1_name'
    }, inplace=True)
    df_worldcities_std['source'] = 'worldcities'
    df_worldcities_std['admin2_name'] = pd.NA
    
    gdf_worldcities_std = gpd.GeoDataFrame(
        df_worldcities_std,
        geometry=gpd.points_from_xy(df_worldcities_std.longitude, df_worldcities_std.latitude),
        crs="EPSG:4326"
    )

    # Combine into a single GeoDataFrame
    cols_to_combine = ['original_id', 'city_name', 'latitude', 'longitude', 'population', 'country_code', 'admin1_name', 'admin2_name', 'geometry', 'source']
    gdf_all_cities = pd.concat([gdf_geonames_std[cols_to_combine], gdf_worldcities_std[cols_to_combine]], ignore_index=True)
    
    gdf_all_cities.drop_duplicates(subset=['city_name', 'latitude', 'longitude'], keep='last', inplace=True)
    for col in ['admin1_name', 'admin2_name']:
        gdf_all_cities[col] = gdf_all_cities[col].apply(normalize_text).fillna('__NONE__')
    print(f"  Combined dataset has {len(gdf_all_cities)} unique cities.")

    # --- Step 2: Project Data and Run HDBSCAN (Unchanged) ---
    print("\nStep 2: Running HDBSCAN clustering...")
    gdf_all_cities_proj = gdf_all_cities.to_crs(epsg=3857)
    coords = np.array(list(zip(gdf_all_cities_proj.geometry.x, gdf_all_cities_proj.geometry.y)))
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size, min_samples=min_samples, metric='euclidean',
        cluster_selection_epsilon=cluster_selection_epsilon_meters or 0
    )
    clusterer.fit(coords)
    gdf_all_cities['cluster_id'] = clusterer.labels_
    print(f"  HDBSCAN found {len(np.unique(clusterer.labels_)) - 1} clusters.")

    # --- Step 3: Identify Representatives for All Three Tiers ---
    print("\nStep 3: Identifying representatives for all three metro tiers...")
    gdf_all_cities['population'] = pd.to_numeric(gdf_all_cities['population'], errors='coerce').fillna(0)
    clustered_cities = gdf_all_cities[gdf_all_cities['cluster_id'] != -1]

    # Tier 1: greater_metro (purely spatial)
    idx_greater = clustered_cities.groupby('cluster_id')['population'].idxmax()
    reps_greater = gdf_all_cities.loc[idx_greater].copy()
    greater_map = reps_greater[['cluster_id', 'city_name', 'original_id']].rename(
        columns={'city_name': 'greater_metro_name', 'original_id': 'greater_metro_id'}
    )
    
    # Tier 2: greater_metro_in_country (spatial, but respects country borders)
    idx_greater_country = clustered_cities.groupby(['cluster_id', 'country_code'])['population'].idxmax()
    reps_greater_country = gdf_all_cities.loc[idx_greater_country].copy()
    greater_country_map = reps_greater_country[['cluster_id', 'country_code', 'city_name', 'original_id']].rename(
        columns={'city_name': 'greater_metro_in_country_name', 'original_id': 'greater_metro_in_country_id'}
    )

    # Tier 3: metro (most granular, respects all admin boundaries)
    idx_local = clustered_cities.groupby(['cluster_id', 'country_code', 'admin1_name', 'admin2_name'])['population'].idxmax()
    reps_local = gdf_all_cities.loc[idx_local].copy()
    local_map = reps_local[['cluster_id', 'country_code', 'admin1_name', 'admin2_name', 'city_name', 'original_id']].rename(
        columns={'city_name': 'metro_name', 'original_id': 'metro_id'}
    )

    # --- Step 4: Map All Tiers Back to the Main DataFrame ---
    print("\nStep 4: Merging three-tier results back into the original 'df' DataFrame...")

    gdf_all_merged = gdf_all_cities.merge(greater_map, on='cluster_id', how='left')
    gdf_all_merged = gdf_all_merged.merge(greater_country_map, on=['cluster_id', 'country_code'], how='left')
    gdf_all_merged = gdf_all_merged.merge(local_map, on=['cluster_id', 'country_code', 'admin1_name', 'admin2_name'], how='left')

    # For noise/unmatched points, they represent themselves in all tiers
    for tier_prefix in ['metro', 'greater_metro_in_country', 'greater_metro']:
        is_unmatched = gdf_all_merged[f'{tier_prefix}_name'].isna()
        gdf_all_merged.loc[is_unmatched, f'{tier_prefix}_name'] = gdf_all_merged.loc[is_unmatched, 'city_name']
        gdf_all_merged.loc[is_unmatched, f'{tier_prefix}_id'] = gdf_all_merged.loc[is_unmatched, 'original_id']
    
    # Filter for the original data source and merge back
    cols_to_merge = [
        'original_id', 'metro_name', 'metro_id', 'greater_metro_in_country_name', 
        'greater_metro_in_country_id', 'greater_metro_name', 'greater_metro_id', 'source'
    ]
    final_cols_to_merge = gdf_all_merged[gdf_all_merged['source'] == 'geonames'][cols_to_merge]
    final_cols_to_merge.drop_duplicates(subset=['original_id'], inplace=True)

    gdf_final = gdf_geonames.merge(
        final_cols_to_merge,
        left_index=True,
        right_on='original_id',
        how='left'
    )
    gdf_final.drop(columns=['original_id'], inplace=True, errors='ignore')
    print(f"final length: {len(gdf_final)}")

    print("\n--- Process Complete! ---")
    return gdf_final

final_gdf_tuned = cluster_cities_with_hdbscan_three_tier(
    df, 
    df2
)

In [ ]:
def cluster_cities_with_hdbscan_three_tier_simplified(
    gdf_all_cities, # The function now takes one pre-merged GeoDataFrame
    min_cluster_size=5,
    min_samples=None,
    cluster_selection_epsilon_meters=None
):
    """
    Identifies a three-tiered metropolitan hierarchy using HDBSCAN on a PRE-MERGED dataset.
    """
    print("--- Starting Three-Tier HDBSCAN Clustering Process ---")

    # --- Step 1: Prepare Data ---
    print("Step 1: Preparing combined data for clustering...")
    gdf_all_cities = gdf_all_cities.copy()
    
    # Standardize column names for processing within this function
    # This ensures consistency even if the input names are slightly different
    gdf_all_cities.rename(columns={
        'city_latitude': 'latitude', 'city_longitude': 'longitude',
        'city_population': 'population', 'city_country_code': 'country_code',
        'city_admin1_name': 'admin1_name', 'city_admin2_name': 'admin2_name'
    }, inplace=True, errors='ignore') # ignore errors if already renamed

    # Normalize admin names and fill NaNs, and add a unique ID
    for col in ['admin1_name', 'admin2_name']:
        gdf_all_cities[col] = gdf_all_cities[col].apply(normalize_text).fillna('')
    gdf_all_cities['original_id'] = gdf_all_cities.index
    print(f"  Processing {len(gdf_all_cities)} total cities.")
    
    # --- Step 2: Project Data and Run HDBSCAN ---
    print("\nStep 2: Running HDBSCAN clustering...")
    gdf_all_cities_proj = gdf_all_cities.to_crs(epsg=3857)
    coords = np.array(list(zip(gdf_all_cities_proj.geometry.x, gdf_all_cities_proj.geometry.y)))
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size, min_samples=min_samples, metric='euclidean',
        cluster_selection_epsilon=cluster_selection_epsilon_meters or 0
    )
    clusterer.fit(coords)
    gdf_all_cities['cluster_id'] = clusterer.labels_
    print(f"  HDBSCAN found {len(np.unique(clusterer.labels_)) - 1} clusters.")

    # --- Step 3: Identify Representatives for All Three Tiers ---
    print("\nStep 3: Identifying representatives for all three metro tiers...")
    gdf_all_cities['population'] = pd.to_numeric(gdf_all_cities['population'], errors='coerce').fillna(0)
    clustered_cities = gdf_all_cities[gdf_all_cities['cluster_id'] != -1]

    idx_greater = clustered_cities.groupby('cluster_id')['population'].idxmax()
    reps_greater = gdf_all_cities.loc[idx_greater].copy()
    greater_map = reps_greater[['cluster_id', 'city_name', 'original_id']].rename(
        columns={'city_name': 'greater_metro_name', 'original_id': 'greater_metro_id'}
    )

    idx_greater_country = clustered_cities.groupby(['cluster_id', 'country_code'])['population'].idxmax()
    reps_greater_country = gdf_all_cities.loc[idx_greater_country].copy()
    greater_country_map = reps_greater_country[['cluster_id', 'country_code', 'city_name', 'original_id']].rename(
        columns={'city_name': 'greater_metro_in_country_name', 'original_id': 'greater_metro_in_country_id'}
    )

    idx_local = clustered_cities.groupby(['cluster_id', 'country_code', 'admin1_name', 'admin2_name'])['population'].idxmax()
    reps_local = gdf_all_cities.loc[idx_local].copy()
    local_map = reps_local[['cluster_id', 'country_code', 'admin1_name', 'admin2_name', 'city_name', 'original_id']].rename(
        columns={'city_name': 'metro_name', 'original_id': 'metro_id'}
    )

    # --- Step 4: Map All Tiers Back to the Main DataFrame ---
    print("\nStep 4: Merging three-tier results back into the DataFrame...")

    gdf_all_merged = gdf_all_cities.merge(greater_map, on='cluster_id', how='left')
    gdf_all_merged = gdf_all_merged.merge(greater_country_map, on=['cluster_id', 'country_code'], how='left')
    gdf_all_merged = gdf_all_merged.merge(local_map, on=['cluster_id', 'country_code', 'admin1_name', 'admin2_name'], how='left')

    for tier_prefix in ['metro', 'greater_metro_in_country', 'greater_metro']:
        is_unmatched = gdf_all_merged[f'{tier_prefix}_name'].isna()
        gdf_all_merged.loc[is_unmatched, f'{tier_prefix}_name'] = gdf_all_merged.loc[is_unmatched, 'city_name']
        gdf_all_merged.loc[is_unmatched, f'{tier_prefix}_id'] = gdf_all_merged.loc[is_unmatched, 'original_id']
    
    # The function now returns the fully merged and clustered dataframe, renaming is undone
    final_cols = ['metro_name', 'metro_id', 'greater_metro_in_country_name', 
                  'greater_metro_in_country_id', 'greater_metro_name', 'greater_metro_id']
    gdf_final = gdf_all_merged.drop(columns=['original_id', 'cluster_id'], errors='ignore').rename(columns={
        'latitude': 'city_latitude', 'longitude': 'city_longitude',
        'population': 'city_population', 'country_code': 'city_country_code',
        'admin1_name': 'city_admin1_name', 'admin2_name': 'city_admin2_name'
    })

    # final cleanup
    gdf_final.insert(loc=1, column='city_name_normalized', value= gdf_final["city_name"].apply(normalize_text)) 
    # Normalize admin names and fill NaNs, and add a unique ID
    for col in ["greater_metro_name",	"greater_metro_in_country_name", "metro_name"]:
        gdf_final[col] = gdf_final[col].apply(normalize_text).fillna('')
    source = gdf_final.pop('source')
    gdf_final.insert(len(gdf_final.columns), 'source', source)

    def create_list_column(row, col1, col2):
        """
        Creates a list from two columns, excluding null values.

        Args:
            row: A row of the DataFrame.
            col1: The name of the first column.
            col2: The name of the second column.

        Returns:
            A list containing non-null values from col1 and col2.
        """
        values = row["alternatenames"]

        # **CRITICAL**: Only add the name if it's a non-empty string.
        # This prevents NaN (float) values from being added to the set.
        if isinstance(row[col1], str) and row[col1]:
            values.append(row[col1])
        if isinstance(row[col2], str) and row[col2]:
            values.append(row[col2])
        
        return list(set(values))

    gdf_final['alternatenames'] = gdf_final.apply(lambda row: create_list_column(row, 'city_name', 'city_name_normalized'), axis=1)

    print(len(gdf_final))

    print("\n--- Process Complete! ---")
    return gdf_final

final_gdf_tuned = cluster_cities_with_hdbscan_three_tier_simplified(
    merged_gdf 
)

# 3. View the results
print(f"Final DataFrame has {len(final_gdf_tuned)} entries.")
final_gdf_tuned.head()